In [1]:
import unittest
import torch
import os.path as osp
import sys

sys.path.insert(0, osp.join('..', 'main'))
sys.path.insert(0, osp.join('..', 'data'))
sys.path.insert(0, osp.join('..', 'common'))
from config import cfg
import argparse

cfg.set_args("0", False)
cfg.num_thread = 1


>>> Using GPU: 0


In [2]:
from model_new_coral import get_model

source_ckpt = r"C:\Users\vladi\RP\InterWild\demo\snapshot_6.pth"
model = get_model(mode='test')
model = torch.nn.DataParallel(model).cuda()

ckpt = torch.load(source_ckpt)
load_info = model.load_state_dict(ckpt['network'], strict=False)

In [2]:
from train_ssa import CustomHandLandmarksDataset
import torchvision.transforms as transforms
val_dataset_target_part1 = CustomHandLandmarksDataset(
    image_dir=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\IR",
    annotations_path=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\combined_FIX_IR.json",
    transform=transforms.ToTensor()
)
val_dataset_target_part2 = CustomHandLandmarksDataset(
    image_dir=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\IR",
    annotations_path=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\IR\landmarks.json",
    transform=transforms.ToTensor()
)


Fix shapedirs bug of MANO


In [3]:
val_dataset_source_part1 = CustomHandLandmarksDataset(
    image_dir=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\RGB", 
    annotations_path=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\combined_FIX_RGB.json",
    transform=transforms.ToTensor()
)
val_dataset_source_part2 = CustomHandLandmarksDataset(
    image_dir=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\RGB", 
    annotations_path=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\RGB\landmarks.json",
    transform=transforms.ToTensor()
)


In [4]:
val_dataset_target_cleaned_part1 = CustomHandLandmarksDataset(
    image_dir=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\IR_copy\allvisible",
    annotations_path=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\combined_FIX_IR.json",
    transform=transforms.ToTensor()
)
val_dataset_target_cleaned_part2 = CustomHandLandmarksDataset(
    image_dir=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\IR_copy\allvisible",
    annotations_path=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\IR\landmarks.json",
    transform=transforms.ToTensor()
)

In [6]:
model.eval()

DataParallel(
  (module): Model(
    (body_backbone): ResNetBackbone(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu

In [5]:
from custom_eval_framework import merge_evaluation_reports
report_target_part1 = val_dataset_target_part1.perform_evaluation(model)
report_target_part2 = val_dataset_target_part2.perform_evaluation(model)
report_target = merge_evaluation_reports(report_target_part1, report_target_part2)

"""
report_source_part1 = val_dataset_source_part1.perform_evaluation(model)
report_source_part2 = val_dataset_source_part2.perform_evaluation(model)
report_source = merge_evaluation_reports(report_source_part1, report_source_part2)
"""
report_target_cleaned_part1 = val_dataset_target_cleaned_part1.perform_evaluation(model)
report_target_cleaned_part2 = val_dataset_target_cleaned_part2.perform_evaluation(model)
report_target_cleaned = merge_evaluation_reports(report_target_cleaned_part1, report_target_cleaned_part2)


NameError: name 'model' is not defined

In [25]:
from custom_eval_framework import merge_evaluation_reports
report_target_part1 = val_dataset_target_part1.perform_evaluation(model, adaptive_threshold=False)
report_target_part2 = val_dataset_target_part2.perform_evaluation(model, adaptive_threshold=False)
report_target_static = merge_evaluation_reports(report_target_part1, report_target_part2)
"""
report_source_part1 = val_dataset_source_part1.perform_evaluation(model, adaptive_threshold=False)
report_source_part2 = val_dataset_source_part2.perform_evaluation(model, adaptive_threshold=False)
report_source_static = merge_evaluation_reports(report_source_part1, report_source_part2)
"""
report_target_cleaned_part1 = val_dataset_target_cleaned_part1.perform_evaluation(model, adaptive_threshold=False)
report_target_cleaned_part2 = val_dataset_target_cleaned_part2.perform_evaluation(model, adaptive_threshold=False)
report_target_cleaned_static = merge_evaluation_reports(report_target_cleaned_part1, report_target_cleaned_part2)


In [26]:
print(report_target)
print(report_target_cleaned)

EvaluationReport:
PCK:
  Mean: 0.585
  Min:  0.000
  Max:  1.000
  Count: 170
IOU:
  Mean: 0.675
  Min:  0.000
  Max:  0.928
  Count: 170
EvaluationReport:
PCK:
  Mean: 0.705
  Min:  0.190
  Max:  1.000
  Count: 47
IOU:
  Mean: 0.763
  Min:  0.154
  Max:  0.928
  Count: 47


In [27]:
print(report_target_static)
print(report_target_cleaned_static)

EvaluationReport:
PCK:
  Mean: 0.741
  Min:  0.024
  Max:  1.000
  Count: 170
IOU:
  Mean: 0.676
  Min:  0.029
  Max:  0.928
  Count: 170
EvaluationReport:
PCK:
  Mean: 0.840
  Min:  0.333
  Max:  1.000
  Count: 47
IOU:
  Mean: 0.763
  Min:  0.154
  Max:  0.928
  Count: 47


In [12]:
print(report_target)
print(report_target_cleaned)

EvaluationReport:
PCK:
  Mean: 0.617
  Min:  0.000
  Max:  1.000
  Count: 170
IOU:
  Mean: 0.712
  Min:  0.000
  Max:  0.935
  Count: 170
EvaluationReport:
PCK:
  Mean: 0.727
  Min:  0.310
  Max:  1.000
  Count: 47
IOU:
  Mean: 0.788
  Min:  0.427
  Max:  0.933
  Count: 47


In [10]:
print(report_target_static)
print(report_target_cleaned_static)

EvaluationReport:
PCK:
  Mean: 0.780
  Min:  0.024
  Max:  1.000
  Count: 170
IOU:
  Mean: 0.713
  Min:  0.041
  Max:  0.935
  Count: 170
EvaluationReport:
PCK:
  Mean: 0.874
  Min:  0.500
  Max:  1.000
  Count: 47
IOU:
  Mean: 0.787
  Min:  0.427
  Max:  0.933
  Count: 47


In [7]:
print(report_target_static)
print(report_target_cleaned_static)

EvaluationReport:
PCK:
  Mean: 0.656
  Min:  0.024
  Max:  1.000
  Count: 170
IOU:
  Mean: 0.627
  Min:  0.051
  Max:  0.935
  Count: 170
EvaluationReport:
PCK:
  Mean: 0.777
  Min:  0.357
  Max:  1.000
  Count: 47
IOU:
  Mean: 0.718
  Min:  0.184
  Max:  0.935
  Count: 47


In [10]:
print(report_source)


EvaluationReport:
PCK:
  Mean: 0.934
  Min:  0.643
  Max:  1.000
  Count: 170
IOU:
  Mean: 0.879
  Min:  0.757
  Max:  0.956
  Count: 170


In [11]:
print(report_target_cleaned)

EvaluationReport:
PCK:
  Mean: 0.650
  Min:  0.238
  Max:  1.000
  Count: 47
IOU:
  Mean: 0.718
  Min:  0.184
  Max:  0.935
  Count: 47


In [18]:
from model_new_coral import get_model

source_ckpt = r"C:\Users\vladi\RP\InterWild\output\model_dump_old\snapshot_0Ach0512.pth"
model = get_model(mode='test')
model = torch.nn.DataParallel(model).cuda()

ckpt = torch.load(source_ckpt)
load_info = model.load_state_dict(ckpt['network'], strict=False)

C:\Users\vladi\AppData\Local\Temp\ipykernel_30908\4063186805.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(source_ckpt)


In [16]:
from custom_eval_framework import merge_evaluation_reports
report_target_part1 = val_dataset_target_part1.perform_evaluation(model)
report_target_part2 = val_dataset_target_part2.perform_evaluation(model)
report_target = merge_evaluation_reports(report_target_part1, report_target_part2)

report_source_part1 = val_dataset_source_part1.perform_evaluation(model)
report_source_part2 = val_dataset_source_part2.perform_evaluation(model)
report_source = merge_evaluation_reports(report_source_part1, report_source_part2)

report_target_cleaned_part1 = val_dataset_target_cleaned_part1.perform_evaluation(model)
report_target_cleaned_part2 = val_dataset_target_cleaned_part2.perform_evaluation(model)
report_target_cleaned = merge_evaluation_reports(report_target_cleaned_part1, report_target_cleaned_part2)


In [17]:
print(report_target)


EvaluationReport:
PCK:
  Mean: 0.533
  Min:  0.024
  Max:  1.000
  Count: 125
IOU:
  Mean: 0.627
  Min:  0.044
  Max:  0.927
  Count: 125


In [18]:
print(report_source)


EvaluationReport:
PCK:
  Mean: 0.938
  Min:  0.571
  Max:  1.000
  Count: 125
IOU:
  Mean: 0.878
  Min:  0.629
  Max:  0.954
  Count: 125


In [19]:
print(report_target_cleaned)

EvaluationReport:
PCK:
  Mean: 0.644
  Min:  0.071
  Max:  1.000
  Count: 37
IOU:
  Mean: 0.704
  Min:  0.044
  Max:  0.923
  Count: 37


In [5]:
from model_new_coral import get_model

source_ckpt = r"C:\Users\vladi\RP\InterWild\output\model_dump_old\snapshot_8_pck535handcoral.pth"
model = get_model(mode='test')
model = torch.nn.DataParallel(model).cuda()

ckpt = torch.load(source_ckpt)
load_info = model.load_state_dict(ckpt['network'], strict=False)

C:\Users\vladi\AppData\Local\Temp\ipykernel_26824\3216114335.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(source_ckpt)


In [21]:
from custom_eval_framework import merge_evaluation_reports
report_target_part1 = val_dataset_target_part1.perform_evaluation(model)
report_target_part2 = val_dataset_target_part2.perform_evaluation(model)
report_target = merge_evaluation_reports(report_target_part1, report_target_part2)

report_source_part1 = val_dataset_source_part1.perform_evaluation(model)
report_source_part2 = val_dataset_source_part2.perform_evaluation(model)
report_source = merge_evaluation_reports(report_source_part1, report_source_part2)

report_target_cleaned_part1 = val_dataset_target_cleaned_part1.perform_evaluation(model)
report_target_cleaned_part2 = val_dataset_target_cleaned_part2.perform_evaluation(model)
report_target_cleaned = merge_evaluation_reports(report_target_cleaned_part1, report_target_cleaned_part2)


In [8]:
print(report_target)


EvaluationReport:
PCK:
  Mean: 0.498
  Min:  0.024
  Max:  1.000
  Count: 170
IOU:
  Mean: 0.623
  Min:  0.051
  Max:  0.935
  Count: 170


In [23]:
print(report_source)


EvaluationReport:
PCK:
  Mean: 0.930
  Min:  0.500
  Max:  1.000
  Count: 125
IOU:
  Mean: 0.857
  Min:  0.627
  Max:  0.946
  Count: 125


In [24]:
print(report_target_cleaned)

EvaluationReport:
PCK:
  Mean: 0.696
  Min:  0.190
  Max:  1.000
  Count: 37
IOU:
  Mean: 0.741
  Min:  0.154
  Max:  0.928
  Count: 37


In [8]:
del model

In [8]:
from model_new_coral import get_model
import torch

# Initialize model
model = get_model(mode='test')
model = torch.nn.DataParallel(model).cuda()

# Load first checkpoint for body parts
body_ckpt_path = r"C:\Users\vladi\RP\InterWild\output\model_dump_old\snapshot_0Ach0512.pth"
body_ckpt = torch.load(body_ckpt_path)
body_state_dict = {k: v for k, v in body_ckpt['network'].items() 
                   if k.startswith(('module.body_backbone', 'module.body_box_net'))}

# Load second checkpoint for hand parts
hand_ckpt_path = r"C:\Users\vladi\RP\InterWild\output\model_dump_old\snapshot_8_pck535handcoral.pth"
hand_ckpt = torch.load(hand_ckpt_path)
hand_state_dict = {k: v for k, v in hand_ckpt['network'].items()
                  if k.startswith(('module.hand_roi_net', 'module.hand_position_net'))}

# Combine state dicts and load
combined_state_dict = {}
combined_state_dict.update(body_state_dict)
combined_state_dict.update(hand_state_dict)

# Load with strict=True to ensure all target keys exist
load_info = model.load_state_dict(combined_state_dict, strict=True)

# Verify loading
print("Missing keys:", load_info.missing_keys)  # Should be empty
print("Unexpected keys:", load_info.unexpected_keys)  # Should be empty

# Save combined model to new checkpoint
save_path = r"C:\Users\vladi\RP\InterWild\output\model_dump_old\combined_model.pth"
torch.save({
    'epoch': 0,  # You can set appropriate values
    'network': model.state_dict(),
    'optimizer': None,  # Add optimizer state if needed
    'meta': {'combined_from': [body_ckpt_path, hand_ckpt_path]}
}, save_path)
print(f"Saved combined model to: {save_path}")

C:\Users\vladi\AppData\Local\Temp\ipykernel_30908\3726844446.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  body_ckpt = torch.load(body_ckpt_path)
C:\Users\vladi\AppDa

Missing keys: []
Unexpected keys: []
Saved combined model to: C:\Users\vladi\RP\InterWild\output\model_dump_old\combined_model.pth


In [6]:
from custom_eval_framework import merge_evaluation_reports
report_target_part1 = val_dataset_target_part1.perform_evaluation(model)
report_target_part2 = val_dataset_target_part2.perform_evaluation(model)
report_target = merge_evaluation_reports(report_target_part1, report_target_part2)

report_source_part1 = val_dataset_source_part1.perform_evaluation(model)
report_source_part2 = val_dataset_source_part2.perform_evaluation(model)
report_source = merge_evaluation_reports(report_source_part1, report_source_part2)

report_target_cleaned_part1 = val_dataset_target_cleaned_part1.perform_evaluation(model)
report_target_cleaned_part2 = val_dataset_target_cleaned_part2.perform_evaluation(model)
report_target_cleaned = merge_evaluation_reports(report_target_cleaned_part1, report_target_cleaned_part2)


In [7]:
print(report_target)


EvaluationReport:
PCK:
  Mean: 0.586
  Min:  0.024
  Max:  1.000
  Count: 125
IOU:
  Mean: 0.648
  Min:  0.025
  Max:  0.967
  Count: 125


In [28]:
print(report_source)


EvaluationReport:
PCK:
  Mean: 0.933
  Min:  0.714
  Max:  1.000
  Count: 125
IOU:
  Mean: 0.855
  Min:  0.732
  Max:  0.958
  Count: 125


In [29]:
print(report_target_cleaned)

EvaluationReport:
PCK:
  Mean: 0.718
  Min:  0.310
  Max:  1.000
  Count: 37
IOU:
  Mean: 0.776
  Min:  0.427
  Max:  0.933
  Count: 37


In [15]:
val_dataset_target = torch.utils.data.ConcatDataset([val_dataset_target_part1, val_dataset_target_part2])

In [6]:
from model_new import get_model

source_ckpt = r"C:\Users\vladi\RP\InterWild\output\model_dump_old\combined_model.pth"
model = get_model(mode='test')
model = torch.nn.DataParallel(model).cuda()

ckpt = torch.load(source_ckpt)
load_info = model.load_state_dict(ckpt['network'], strict=False)

C:\Users\vladi\AppData\Local\Temp\ipykernel_26824\4083501755.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(source_ckpt)


In [13]:
model.eval()
# Put all BatchNorm layers in train mode while keeping the model in eval mode
for module in model.modules():
    if isinstance(module, torch.nn.BatchNorm2d):
        module.train()
        module.momentum = 0.01


In [16]:
val_loader = torch.utils.data.DataLoader(val_dataset_target, batch_size=32, shuffle=False)

for batch in val_loader:
    source_inputs, source_targets, source_meta_info = batch
    for k in source_inputs:
        source_inputs[k] = source_inputs[k].cuda()
    for k in source_targets:
        source_targets[k] = source_targets[k].cuda()
    for k in source_meta_info:
        # If meta_info holds Tensors, also send to device
        # Otherwise skip
        if isinstance(source_meta_info[k], torch.Tensor):
            source_meta_info[k] = source_meta_info[k].cuda()

    outputs = model.forward(
        source_inputs,
        source_targets,
        source_meta_info,  # Pass combined dictionary as meta_info
        "test",  # Pass 'train' as the mode
    )


Entered __getitem__
[492. 364. 822. 642.] [494. 710. 828. 994.]
Entered __getitem__
[458. 416. 808. 676.] [ 452.  786.  796. 1058.]
Entered __getitem__
[438. 396. 784. 642.] [ 420.  746.  784. 1020.]
Entered __getitem__
[466. 316. 804. 544.] [468. 654. 814. 934.]
Entered __getitem__
[476. 358. 820. 588.] [466. 684. 806. 956.]
Entered __getitem__
[348. 300. 708. 628.] [ 308.          720.          694.66666667 1066.66666667]
Entered __getitem__
[345.33333333 324.         721.33333333 670.66666667] [ 341.33333333  761.33333333  734.66666667 1101.33333333]
Entered __getitem__
[398.66666667 318.66666667 793.33333333 632.        ] [ 406.66666667  677.33333333  792.         1034.66666667]
Entered __getitem__
[398.66666667 294.66666667 778.66666667 608.        ] [ 382.66666667  722.66666667  766.66666667 1078.66666667]
Entered __getitem__
[381.33333333 265.33333333 753.33333333 572.        ] [ 425.33333333  708.          801.33333333 1005.33333333]
Entered __getitem__
[496. 410. 802. 686.] [ 

In [17]:
# Free up CUDA memory
torch.cuda.empty_cache()


In [18]:
report_target_part1 = val_dataset_target_part1.perform_evaluation(model, model_type="classic")
report_target_part2 = val_dataset_target_part2.perform_evaluation(model, model_type="classic")
report_target = merge_evaluation_reports(report_target_part1, report_target_part2)

[471.05744361877436, 435.5885982513428, 817.8269004821777, 689.2771625518799] [494.0, 710.0, 828.0, 994.0]
[464.3275451660156, 478.3582019805908, 794.5772552490234, 720.7719612121582] [452.0, 785.9999999999999, 796.0, 1058.0]
[447.91500091552734, 442.42509841918945, 785.9622573852539, 696.7115592956543] [420.0, 746.0000000000001, 784.0, 1020.0]
[461.24613761901855, 644.9626922607422, 752.1302032470703, 880.1889038085938] [468.0, 654.0, 814.0, 934.0]
[467.6031875610351, 677.5645351409912, 748.10302734375, 882.9944515228271] [466.0, 684.0, 806.0, 956.0]
[319.67519760131836, 523.6223888397217, 672.4281692504883, 902.8899192810059] [308.0, 720.0, 694.6666666666666, 1066.6666666666665]
[336.70263290405273, 377.6111698150635, 723.3352947235107, 690.9047698974609] [341.3333333333333, 761.3333333333333, 734.6666666666666, 1101.3333333333333]
[420.55277824401855, 669.8623466491699, 778.1089210510253, 1026.400966644287] [406.6666666666667, 677.3333333333334, 792.0, 1034.6666666666667]
[377.96650

In [19]:
print(report_target)

EvaluationReport:
PCK:
  Mean: 0.616
  Min:  0.000
  Max:  1.000
  Count: 170
IOU:
  Mean: 0.386
  Min:  0.000
  Max:  0.813
  Count: 170


In [11]:
from custom_eval_framework import merge_evaluation_reports
report_target = merge_evaluation_reports(report_target_part1, report_target_part2)

In [12]:
print(report_target)

EvaluationReport:
PCK:
  Mean: 0.617
  Min:  0.000
  Max:  1.000
  Count: 170
IOU:
  Mean: 0.372
  Min:  0.000
  Max:  0.813
  Count: 170
